In [1]:
import json
import pandas as pd
import requests
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py

In [2]:
db_name = 'MacroData2'

In [3]:
def get_BLS_data(seriesIDs, startyear, endyear):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = { 
        "seriesid":seriesIDs,
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        #message = "For series " + seriesIDs + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [4]:
def check_table_exists(tbl_name, db_name): 
    """
    Check if table exists
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    exists = 0
    c.execute("SELECT count(name) FROM sqlite_master WHERE type='table'")
    if c.fetchone()[0] == 0:
        print("No tables in DB.")
    else:
        try:
            c.execute("SELECT COUNT(name) FROM sqlite_master WHERE type = 'table' AND name='" + tbl_name +"';")
        except Error as e:
            print(e)
        if c.fetchone()[0] == 1:
            exists = 1
        
    conn.close()
    
    return exists


def create_data_table(tbl_name, db_name):
    """
    create data table to hold series values
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code1 = ("""CREATE TABLE IF NOT EXISTS """ + tbl_name + """ (
                    date PRIMARY KEY
                    ,latest INT
                    ,value
                    ,series FORIEGN KEY
                ) WITHOUT ROWID;""")
    
    conn_execute(db_name, sql_code1)
    '''if not check_table_exists(tbl_name ,db_name):
        try: 
            input_df.to_sql(tbl_name, conn)
            print("Created ", tbl_name, " table.")
        except ValueError as e:
            print(e)
            print("Table was not added to the DB")'''    
    
    conn.close()
    
def create_dict_table(tbl_name, db_name):
    """
    create dictionary table to hold the attributes of the series
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code1 = ("""CREATE TABLE IF NOT EXISTS """ + tbl_name + """ (
                    series_title
                    ,series_id PRIMARY KEY
                    ,seasonality
                    ,survey_long_name
                    ,survey_short_name
                    ,survey_abbreviation
                    ,measure_data_type
                    ,area
                    ,item
                ) WITHOUT ROWID;""")
    
    conn_execute(db_name, sql_code1)
    '''if not check_table_exists(tbl_name ,db_name):
        try: 
            input_df.to_sql(tbl_name, conn)
            print("Created ", tbl_name, " table.")
        except ValueError as e:
            print(e)
            print("Table was not added to the DB")'''    
    
    conn.close()
    
def add_data_to_db(df_EntryData, tbl_name, db_name):  
    """
    Add dataframe to 
    database: db_name 
    table: tbl_name
    """
    conn = create_connection(db_name)    
    try:
        df_EntryData.to_sql(tbl_name, con=conn, if_exists = 'append')
        print("Data added to DB: \n", df_EntryData)
    except ValueError as e:
        print(e)
        print("Data was not added to the DB")
    conn.close()
    

def table_to_df(tbl_name, db_name):
    conn = db.connect(db_name)
    try:
        df_fromDB = pd.read_sql_query("SELECT * FROM " + tbl_name + ";", conn)#, index_col="date")
    except Error as e:
        print(e)
    
    conn.close()
    
    return df_fromDB

In [5]:
# CUUR0000SA0: 
# CUSR0000SA0
seriesIDs = ["CUUR0000SA0", "CUSR0000SA0"] 
message, json_data = get_BLS_data(seriesIDs, 2004, 2023)
if message:
    print(message) 

In [6]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 413,
 'message': [],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'catalog': {'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
     'series_id': 'CUUR0000SA0',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'CPI for All Urban Consumers (CPI-U)',
     'survey_abbreviation': 'CU',
     'measure_data_type': 'All items',
     'area': 'U.S. city average',
     'item': 'All items'},
    'data': [{'year': '2023',
      'period': 'M01',
      'periodName': 'January',
      'latest': 'true',
      'value': '299.170',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '296.797',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '297.711',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': 'October',

In [15]:
# Get number of series
number_of_series = len(json_data['Results']['series'])
number_of_series

2

In [17]:
for i in range(len(json_data['Results']['series'])):
    df_dict = pd.DataFrame()
    df_dict['series_title'] = json_data['Results']['series'][0]['catalog']['series_title'],
    df_dict['series_id'] = json_data['Results']['series'][0]['catalog']['series_id'],
    df_dict['seasonality'] = json_data['Results']['series'][0]['catalog']['seasonality'],
    df_dict['survey_long_name'] = json_data['Results']['series'][0]['catalog']['survey_name'],
    df_dict['survey_short_name'] = json_data['Results']['series'][0]['catalog']['survey_name'][-6:-1],
    df_dict['survey_abbreviation'] = json_data['Results']['series'][0]['catalog']['survey_abbreviation'],
    df_dict['measure_data_type'] = json_data['Results']['series'][0]['catalog']['measure_data_type'],
    df_dict['area'] = json_data['Results']['series'][0]['catalog']['area'],
    df_dict['item'] = json_data['Results']['series'][0]['catalog']['item']
    df_dict = df_dict.set_index(df_dict['series_id'])
    df_dict = df_dict.drop(columns=['series_id'])

0
1


In [10]:
df_dict = pd.DataFrame()
df_dict['series_title'] = json_data['Results']['series'][0]['catalog']['series_title'],
df_dict['series_id'] = json_data['Results']['series'][0]['catalog']['series_id'],
df_dict['seasonality'] = json_data['Results']['series'][0]['catalog']['seasonality'],
df_dict['survey_long_name'] = json_data['Results']['series'][0]['catalog']['survey_name'],
df_dict['survey_short_name'] = json_data['Results']['series'][0]['catalog']['survey_name'][-6:-1],
df_dict['survey_abbreviation'] = json_data['Results']['series'][0]['catalog']['survey_abbreviation'],
df_dict['measure_data_type'] = json_data['Results']['series'][0]['catalog']['measure_data_type'],
df_dict['area'] = json_data['Results']['series'][0]['catalog']['area'],
df_dict['item'] = json_data['Results']['series'][0]['catalog']['item']
df_dict = df_dict.set_index(df_dict['series_id'])
df_dict = df_dict.drop(columns=['series_id'])
df_dict

,series_title,seasonality,survey_long_name,survey_short_name,survey_abbreviation,measure_data_type,area,item
series_id,,,,,,,,
CUUR0000SA0,"All items in U.S. city average, all urban cons...",Not Seasonally Adjusted,CPI for All Urban Consumers (CPI-U),CPI-U,CU,All items,U.S. city average,All items


In [11]:
df = pd.DataFrame(json_data['Results']['series'][0]['data'])
df['series'] = json_data['Results']['series'][0]['catalog']['series_id']
df

,year,period,periodName,latest,value,footnotes,series
0,2023,M01,January,true,299.170,[{}],CUUR0000SA0
1,2022,M12,December,NaN,296.797,[{}],CUUR0000SA0
2,2022,M11,November,NaN,297.711,[{}],CUUR0000SA0
3,2022,M10,October,NaN,298.012,[{}],CUUR0000SA0
4,2022,M09,September,NaN,296.808,[{}],CUUR0000SA0
...,...,...,...,...,...,...,...
224,2004,M05,May,NaN,189.1,[{}],CUUR0000SA0
225,2004,M04,April,NaN,188.0,[{}],CUUR0000SA0
226,2004,M03,March,NaN,187.4,[{}],CUUR0000SA0
227,2004,M02,February,NaN,186.2,[{}],CUUR0000SA0


In [12]:
df['month'] = df['period'].str.replace('M', '')
df['date'] = df['year'] + df['month']
df = df.sort_values(by=['date'], ignore_index=True)
df['date'] = pd.to_numeric(df['date'])
df = df.set_index(df["date"])
df['latest'].iloc[-1] = df['latest'].iloc[-1].upper()
df = df.drop(columns=['year', 'period', 'periodName', 'footnotes', 'month', 'date'])
df

C:\Users\Joshua\AppData\Local\Temp\ipykernel_1964\2745341384.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['latest'].iloc[-1] = df['latest'].iloc[-1].upper()


,latest,value,series
date,,,
200401,NaN,185.2,CUUR0000SA0
200402,NaN,186.2,CUUR0000SA0
200403,NaN,187.4,CUUR0000SA0
200404,NaN,188.0,CUUR0000SA0
200405,NaN,189.1,CUUR0000SA0
...,...,...,...
202209,NaN,296.808,CUUR0000SA0
202210,NaN,298.012,CUUR0000SA0
202211,NaN,297.711,CUUR0000SA0


In [13]:
series_id = json_data['Results']['series'][0]['catalog']['series_id']
create_data_table(series_id, db_name)
add_data_to_db(df, series_id, db_name)
check_df = table_to_df(series_id, db_name)
check_df

Data added to DB: 
        latest    value       series
date                               
200401    NaN    185.2  CUUR0000SA0
200402    NaN    186.2  CUUR0000SA0
200403    NaN    187.4  CUUR0000SA0
200404    NaN    188.0  CUUR0000SA0
200405    NaN    189.1  CUUR0000SA0
...       ...      ...          ...
202209    NaN  296.808  CUUR0000SA0
202210    NaN  298.012  CUUR0000SA0
202211    NaN  297.711  CUUR0000SA0
202212    NaN  296.797  CUUR0000SA0
202301   TRUE  299.170  CUUR0000SA0

[229 rows x 3 columns]


,date,latest,value,series
0,200401,None,185.2,CUUR0000SA0
1,200402,None,186.2,CUUR0000SA0
2,200403,None,187.4,CUUR0000SA0
3,200404,None,188.0,CUUR0000SA0
4,200405,None,189.1,CUUR0000SA0
...,...,...,...,...
224,202209,None,296.808,CUUR0000SA0
225,202210,None,298.012,CUUR0000SA0
226,202211,None,297.711,CUUR0000SA0
227,202212,None,296.797,CUUR0000SA0


In [14]:
series_dict = series_id + "_dict"
create_dict_table(series_dict, db_name)
add_data_to_db(df_dict, series_dict, db_name)
check_df = table_to_df(series_dict, db_name)
check_df

Data added to DB: 
                                                   series_title  \
series_id                                                        
CUUR0000SA0  All items in U.S. city average, all urban cons...   

                         seasonality                     survey_long_name  \
series_id                                                                   
CUUR0000SA0  Not Seasonally Adjusted  CPI for All Urban Consumers (CPI-U)   

            survey_short_name survey_abbreviation measure_data_type  \
series_id                                                             
CUUR0000SA0             CPI-U                  CU         All items   

                          area       item  
series_id                                  
CUUR0000SA0  U.S. city average  All items  


,series_title,series_id,seasonality,survey_long_name,survey_short_name,survey_abbreviation,measure_data_type,area,item
0,"All items in U.S. city average, all urban cons...",CUUR0000SA0,Not Seasonally Adjusted,CPI for All Urban Consumers (CPI-U),CPI-U,CU,All items,U.S. city average,All items


## Validate Data to Put into Correct Table

In [ ]:
 

    
def create_insert_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_insert 
                    AFTER INSERT ON """ + seriesID + """ 
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    
    conn_execute((sql_code)
    
    conn.close()    

def create_update_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_update 
                    AFTER UPDATE ON """ + seriesID + """
                    WHEN old.value <> new.value
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    conn_execute((sql_code)
    
    conn.close()

'''def unique_index(tbl_name, db_name)
    """
    Create a unique index for the date value of the table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = 'CREATE UNIQUE INDEX idx_date ON ' + tbl_name + '(date)';
    
    conn_execute(sql_code)'''
    
def validate_df(df, tbl_name, db_name):
    """
    check if the data from the api df already exists in the table and 
    if the data in the table is duplicate or inconsistent
    return dataframe df without duplicate data or inconsistent data
    return dataframe df_dup_data with duplicate date
    return dataframe df_inconsistent_data with inconsistent data
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    # Set up empty dataframes to use to store data temporarily
    df_not_dup_data = pd.DataFrame(columns = list(df.columns))
    df_dup_data = pd.DataFrame(columns = list(df.columns))
    df_inconsistent_data = pd.DataFrame(columns = list(df.columns))
    
    if check_table_exists(seriesID, db_name) != 0:
        # Check if data already exists in DB
        for i, row in df.iterrows():
            c.execute("SELECT date, value FROM " + tbl_name + " WHERE date = ?;", (df.loc[i]["date"],))
            queryOut = c.fetchall()
            
            # Data does not already exist in the database 
            if len(queryOut) == 0: 
                df_not_dup_data = pd.concat([df_not_dup_data, df.loc[[i]]])
            # Data already exists in the database with duplicate values
            elif queryOut[0][1] == df.loc[i]["value"]: 
                df_dup_data = pd.concat([df_dup_data, df.loc[[i]]])
            # Data already exists in the database but with different values
            elif queryOut[0][1] != df.loc[i]["value"]:
                df_inconsistent_data = pd.concat([df_inconsistent_data, df.loc[[i]]])            
    else:
        df_not_dup_data = df.copy()
    
    conn.close()
    
    df_not_dup_data = df_not_dup_data.sort_values(by=['date'])
    
    return df_not_dup_data, df_dup_data, df_inconsistent_data



    
    
def get_max_index(tbl_name, db_name):
    '''
    Get the maximum index value of the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT MAX([index]) FROM " + seriesID)
        max_index = c.fetchone()[0]
        return max_index
    except Error as e:
        print(e)
    finally:
        conn.close()
        
def get_max_date(tbl_name, db_name):
    '''
    Get the maximum date from the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT date FROM " + seriesID + " WHERE [index] = (SELECT MAX([index]) FROM " + seriesID +")")
        max_date = c.fetchone()[0]
        return max_date
    except Error as e:
        print(e)
    finally:
        conn.close()




